Lorenzo Corrado - Federico Zanotti

# Zeroth Order Methods for Adversarial Machine Learning
### Optimization for Data Science Project








## Import and Data preparation


In [ ]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from keras.optimizers import Optimizer
from keras.models import Model
from scipy.special import softmax
from scipy.optimize import minimize
from scipy.special import softmax
from tqdm import tqdm
import random
import gzip
import urllib.request
import time
import warnings
import os
from prettytable import PrettyTable


# Remove warnings in output
warnings.filterwarnings("ignore")

# Set seed
np.random.seed([2021])

In [ ]:
# Download MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")

# Model/data parameters
num_classes = 10
input_shape = (784,)

# Scale images in [0,1] range
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255

# Make sure images have shape (28,28,1)
# x_train = np.expand_dims(x_train, -1)
# x_test = np.expand_dims(x_test, -1)

x_train = x_train.reshape((60000, 784))
x_test = x_test.reshape((10000, 784))

# Splitting training set in training and valid
validation_size = 5000

x_val = x_train[:validation_size]
y_val = y_train[:validation_size]
x_train = x_train[validation_size:]
y_train = y_train[validation_size:]

# One-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define DNN architecture
model = keras.Sequential()
model.add(tf.keras.Input(shape=input_shape))
model.add(tf.keras.layers.Reshape((28,28,1), input_shape=input_shape))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(num_classes, name="last_dense"))
model.add(tf.keras.layers.Activation("softmax"))

In [ ]:
# Take the output of the last layer before the softmax operation
dnn = Model(inputs=model.input, outputs=model.get_layer("last_dense").output)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

## github link: https://github.com/IBM/ZOSVRG-BlackBox-Adv/tree/master/models
## Loads the weights
path = "./mnist"
if os.path.isfile(path):
  model.load_weights(path) 

else:
  print(f"file {path} does not exist")

## Utility Function

In [ ]:
# Definition of the objective function to be minimized #(FOR ALL THE SAMPLES)
def F(x, y_true):
  """
  Loss function for all the examples

  Input:
  - x: images
  - y_true: true labels of the images

  """
  f = dnn.predict(x)
  f_yi = np.max(f*y_true, axis=1)
  f_j = np.max(f*np.where(y_true == 1, -1e10, 1), axis=1)
  
  return np.mean(np.where(f_yi - f_j > 0, f_yi - f_j, 0))

def F_Par(x, y_true):
  """
  Loss function for only one example

  Input:
  - x: image
  - y: true label of the image

  """
  f = dnn.predict(x)
  f_yi = np.max(f*y_true, axis=1)
  f_j = np.max(f*np.where(y_true == 1, -1e10, 1), axis=1)

  return np.where(f_yi-f_j > 0, f_yi-f_j, 0)

In [ ]:
# Extract n images from the same class
def extract_images(n, c):
  """
  Extract some images of the same class

  Input:
  - n: number of images to extract
  - c: label

  """
  x_extr = np.copy(x_test[y_test.argmax(axis=1)==c][:n])
  y_extr = np.copy(y_test[y_test.argmax(axis=1)==c][:n])
  
  return x_extr, y_extr

In [ ]:
def get_data(n, c):
  """

  Get n images of same class 

  Input:
  -n: sample size
  -c: label of image

  """
  img_in, y_true_in = extract_images(n,c)
  x_ori = np.copy(img_in)
  x = np.copy(x_ori)
  return x, x_ori, y_true_in 

In [ ]:
def stop_attack(x, y_true):
  """ 
  Stopping criterion

  Input: 
  - x: images
  - y_true: true label of images

  """
  success = dnn.predict(x).argmax(axis=1)
  return sum(success==y_true.argmax(axis=1))==0


In [ ]:
def RandGradEst(x, y_true, v):
  """
  Two-point (gaussian) random gradient estimator

  Input:
  - x: image
  - y_true: true label of the image
  - v: smoothing parameter 
  """
  u = np.random.standard_normal((1,d))
  F_plus = F(x + v*u, y_true)
  F_ = F(x, y_true)
  
  return (d/v)*(F_plus - F_)*u

In [ ]:
def Avg_RandGradEst(x, y_true, q, v):
  """
  Averaged (gaussian) random gradient estimator

  Input:
  - x: image
  - y_true: true label of the image
  - q: number of random directions
  - v: smoothing parameter
  """
  g = 0
  u = np.random.standard_normal((q,d))
  F_ = F(x, y_true)
  for j in range(q):
    F_plus = F(x + v*u[j], y_true)
    g = g + (F_plus - F_)*u[j]

  return (d/(v*q))*g

In [ ]:
def CoordGradEst(x, y_true, mu):
  """
  Coordinate-wise gradient estimator
  
  Input:
  - x: images
  - y_true: true labels of the images
  - mu: smoothing parameter

  """
  q = 0
  for j in tqdm(range(d)):
    F_plus = F(x + mu*e(j,d), y_true)
    F_minus = F(x - mu*e(j,d), y_true)
    diff = F_plus - F_minus
    q = q + (diff)*e(j,d)
   
  return q/(2*mu)

In [ ]:
def attack_success_rate(x, y):
  """
  Calculate in percentage the Attack Success Rate of x

  Input:
  - x: images
  - y: true labels of images
  """
  predicted=softmax(dnn.predict(x)).argmax(axis=1)
  # print(predicted)
  true_values=y.argmax(axis=1)
  adversarial=len(predicted)-sum(predicted==true_values)
  wrong_label=(adversarial/len(predicted))*100
  return round(wrong_label,1)


In [ ]:
def plot_all(loss_ZSCG, loss_FZFW, loss_FZCGS, epochs, n, title="",savefig=""):
  """
  Utility function to plot the loss of three algorithms
  
  Input:
  - loss_ZSCG: loss of methods ZSCG
  - loss_FZFW: loss of methods FZFW
  - loss_FZSCG: loss of methods FZSCG
  - epochs: number of iterations
  - n: sample size

  """
  plt.figure(figsize=(10,8))
  plt.plot(loss_ZSCG, label=f"ZSCG with {n} examples")
  plt.plot(loss_FZFW, label=f"FZFW with {n} examples")
  plt.plot(loss_FZCGS, label=f"FZCGS with {n} examples")
  plt.grid("on")
  plt.legend()
  plt.xticks(np.arange(0,epochs+10,10))
  plt.xlabel("# iterations")
  plt.ylabel("loss")
  if title!="":
    plt.title(title)
  if savefig != "":
    plt.savefig(savefig)
  plt.show()



In [ ]:
def nice_table(x_mod, y_true, param1, param2):
  """
  Utility function to visualize a nice plot for the grid search of Hyperparams
  
  Input:
  - x_mod: images perturbated
  - y_true: true label of images
  - param1: (name of param1, value of param1)
  - param1: (name of param2, value of param2)
  """
  param1_list = param1[1]
  param2_list = param2[1]
  j=0
  i=0
  t= PrettyTable([param1[0], param2[0], "ASR (%)"])
  for el in x_mod:
    asr=attack_success_rate(el, y_true)
    if i==len(param2_list):
      i=0
      p2 = param2_list[i]
      j +=1
      p1=param1_list[j]
    else:
      p2 = param2_list[i]
      p1=param1_list[j]
    i +=1
    
    t.add_row([f" {p1}", f" {p2}", asr])
  return t

## Algorithm 1. Zeroth-Order Stochastic Conditional Gradient Method (ZSCG)

K. Balasubramanian et al., 2018.

In [ ]:
def Avg_RandGradEst_Par(x, y_true, q, v, d):
  """
  Averaged (gaussian) random gradient estimator in parallel

  Input:
  - x: image
  - y_true: true label of the image
  - q: number of random directions
  - v: smoothing parameter
  """
  g = 0
  u = np.random.standard_normal((q,d))
  x_par_plus = np.array([x + v*u[j] for j in range(q)]).reshape((q,d))
  diff = F_Par(x_par_plus, y_true) - F(x, y_true)

  for j in range(q):
    g = g + (diff[j]/v) * u[j] 
  
  return (d/(q*v))*g

In [ ]:
def ZSCG(N, d, s, m_k, x, y_true_in,v=-1,alpha=-1, B=1,verbose=True, clip=False):
  """

  Zeroth-Order Stochastic Gradient Method

  Input:
  - N: number of epochs
  - d: dimension of data
  - s: distorsion
  - x: images
  - y_true_in: true label of images
  - v: smoothing parameter 
  - alpha: step size
  - B: parameter of convergence theory
  - verbose: True/False for printing computations
  - clip: True/False activate clipping
  
  """
  if v==-1:
    v = np.sqrt(2/(N*(d+3)**3))
  if alpha==-1:
    alpha = 1/np.sqrt(N)

  x_ori=np.copy(x)
  loss_ZSCG = []
  perturbations = []
  loss_ZSCG.append(F(x, y_true_in))
  if verbose:
    print("Epoch:", 0, "Loss:", F(x_ori, y_true_in), "Distortion:", np.max(np.abs(x-x_ori)))
  for k in range(N):
      
    # Get the gradient estimate
    v_k = 0
    for i in tqdm(range(x.shape[0]), disable= not verbose):
      v_k = v_k + Avg_RandGradEst_Par(x[i:i+1], y_true_in[i:i+1], m_k, v, d)    
    v_k = (1/n)*v_k

    x_k = -s * np.sign(v_k) + x_ori # Solve the LMO
    x = (1 - alpha)*x+ alpha*x_k
    if clip:
      x= x_ori + np.clip((x-x_ori), 0, 1)

    perturbations.append(x)
    loss_ZSCG.append(F(x, y_true_in))

    if verbose:
      print("-"*100)
      print("Epoch:", k+1, "Loss:", loss_ZSCG[k], "Distortion:", np.round(np.max(np.abs(x-x_ori)),5), "Elapsed Time:")

    if stop_attack(x, y_true_in):
      print("Attack successful! stopping computation...")
      return loss_ZSCG, x

  ZSCG_x_perturbated = x

  print("ZSCG Final loss = ", loss_ZSCG[-1])
  return loss_ZSCG, ZSCG_x_perturbated, perturbations

## Algorithm 2. Faster Zeroth-Order Frank-Wolfe Method (FZFW)
Gao et al., 2020.

In [ ]:
def e(i, d):
  """
  Orthogonal basis vector

  Input:
  - i: index
  - d: dimensions
  
  """
  ei = np.zeros(d)
  ei[i] = 1
  return ei

In [ ]:
def CoordGradEst_Par(x, y_true, mu,d):
  """
  Coordinate-wise gradient estimator in parallel

  Input:
  - x: image
  - y_true: true label of the image
  - mu: smoothing parameter
  
  """
  x_par_plus = np.array([x + mu*e(j,d) for j in range(d)]).reshape(d,d)
  x_par_minus = np.array([x - mu*e(j,d) for j in range(d)]).reshape(d,d)
  diff = F_Par(x_par_plus, y_true) - F_Par(x_par_minus, y_true)
  
  q = 0
  for j in range(d):
    q = q + (diff[j])*(e(j,d)) 
    
  return (1/(2*mu))*q

In [ ]:
def FZFW(K,d,n,s,gamma, mu,x,y_true_in, verbose=True, clip=False): 
  """
  Faster Zeroth-Order Frank-Wolfe Method
  
  Input:
  - K: number of epochs
  - d: dimension of data
  - n: sample size
  - s: distorsion
  - gamma: step size 
  - mu: smoothing parameter 
  - x: images
  - y_true_in: true label of images
  - verbose: True/False for printing computations
  - clip: True/False activate clipping
  
  """
  s_1=n
  q = s_2 = int(np.sqrt(n))
  if gamma==-1:
    gamma = 1/np.sqrt(K)
  if mu==-1:
    mu = 1/np.sqrt(d*K)
  x_ori=np.copy(x)


  loss_FZFW = []
  perturbations=[]
  loss_FZFW.append(F(x_ori, y_true_in))
  if verbose:
    print("Epoch:", 0, "Loss:", F(x_ori, y_true_in), "Distortion:", np.max(np.abs(x-x_ori)))
    print("-"*100)


  for k in range(K):
    if (k % q == 0):

      v_k = 0
      for i in tqdm(range(s_1), disable= not verbose):
        v_k = v_k + CoordGradEst_Par(x[i:i+1], y_true_in[i:i+1], mu,d)
      v_k=v_k/s_1
      v_k_1 = v_k

    else:



      v_k = 0
      s2_idx = np.random.randint(0, n, s_2) 

      for idx in tqdm(s2_idx,  disable= not verbose):
        v_k = v_k + CoordGradEst_Par(x[idx:idx+1], y_true_in[idx:idx+1], mu,d) - CoordGradEst_Par(x_k_1[idx:idx+1], y_true_in[idx:idx+1], mu,d) + v_k_1
      v_k = (1/s_2) * v_k
      v_k_1 = v_k



    x_k_1 = np.copy(x)
    u_k = -s * np.sign(v_k) + x_ori # Solve the LMO
    d_k = u_k - x
    x = x + gamma*d_k


    if clip:
      x= x_ori + np.clip((x-x_ori), 0, 1)
    
    perturbations.append(x)
    loss_FZFW.append(F(x, y_true_in))

    if verbose:
      print("Epoch:", k+1, "Loss:", loss_FZFW[k+1], "Distortion:", np.round(np.max(np.abs(x-x_ori)),5))
      print("-"*100)

    if stop_attack(x, y_true_in):
      print("Attack successful! stopping computation...")
      return loss_FZFW, x
    

 
  FZFW_x_perturbated = x

  print("FZFW Final loss = ", loss_FZFW[-1])

  return loss_FZFW, FZFW_x_perturbated, perturbations


## Algorithm 3. Faster Zeroth-Order Conditional Gradient Sliding Method (FZCGS)

Gao et al., 2020.

In [ ]:
def CondG(g, u, gamma, eta, x_ori, s, d):
  """
  Conditional Gradient Sliding Algorithm
  
  Input:
  - g: gradient
  - u: images perturbated
  - eta: threshold parameter
  - x_ori: images without perturbations
  - s: distorsion
  - d: dimension of images

  """
  t=1
  u_t=u
  
  while (True):
    grad = g + (1/gamma)*(u_t-u)
    v = -s * np.sign(g) + x_ori
    V = np.dot(grad, (u_t-v).T)
    if (np.max(V) <= eta):
      return u_t
    if t==100:
      return u_t

    else:
      
      d = v-u_t
      alpha = np.dot((1/gamma) * (u-u_t)-g, d.T) / ((1/gamma)*np.dot(d, d.T))
      alpha = np.min(alpha) 
      if (alpha > 1):
        alpha = 1     
      u_t = (1-alpha)*u_t + alpha*v
      t = t+1
  


In [ ]:
def FZCGS(K,d,n,s,gamma, mu, eta, x,y_true_in, verbose=True, clip=False):
  """
  Faster Zeroth-Order Conditional Gradient Method
  
  Input:
  - K: number of epochs
  - d: dimension of data
  - n: sample size
  - s: distorsion
  - gamma: step size 
  - mu: smoothing parameter 
  - eta: treshold parameter 
  - x: images
  - y_true_in: true label of images
  - verbose: True/False for printing computations
  - clip: True/False activate clipping
  
  """
  s_1 = n
  q = s_2 = int(np.sqrt(n))
  if gamma==-1:
    gamma = 1/np.sqrt(K)
  if mu==-1:
    mu = 1/np.sqrt(d*K)
  if eta==-1:
    eta = 1/K
  x_ori=np.copy(x)


  loss_FZCGS = []
  perturbations=[]
  loss_FZCGS.append(F(x_ori, y_true_in))
  if verbose:
    print("Epoch:", 0, "Loss:", F(x_ori, y_true_in), "Distortion:", np.max(np.abs(x-x_ori)))
    print("-"*100)

  for k in range(K):
    
    if (k % q == 0):
      

      v_k = 0
      for i in tqdm(range(s_1), disable=not verbose):
        v_k = v_k + CoordGradEst_Par(x[i:i+1], y_true_in[i:i+1], mu,d)
      v_k=v_k/s_1
      v_k_1 = v_k

    else:
    

      v_k = 0
      s2_idx = np.random.randint(0, n, s_2) 
    
      for idx in tqdm(s2_idx,  disable= not verbose):
        # idx = np.random.randint(0,n)
        v_k = v_k + CoordGradEst_Par(x[idx:idx+1], y_true_in[idx:idx+1], mu, d) - CoordGradEst_Par(x_k_1[idx:idx+1], y_true_in[idx:idx+1], mu,d) + v_k_1
      v_k = (1/s_2) * v_k
      v_k_1 = v_k


    x_k_1 = np.copy(x)
    x = CondG(v_k, x, gamma, eta, x_ori, s, d) # Cong procedure


    if clip:
      x= x_ori + np.clip((x-x_ori), 0, 1)
    perturbations.append(x)
    loss_FZCGS.append(F(x, y_true_in))
    if verbose:
      print("-"*100)
      print("Epoch:", k+1, "Loss:", loss_FZCGS[k], "Distortion:", np.round(np.max(np.abs(x-x_ori)),5))
    if stop_attack(x, y_true_in):
      print("Attack successful! stopping computation...")
      return loss_FZCGS, x
    


  FZCGS_x_perturbated = x

  print("FZCGS Final loss = ", loss_FZCGS[-1])

  return loss_FZCGS, FZCGS_x_perturbated, perturbations